# Dummy Example

This notebook shows the use of the dummy feedback function provider which
behaves like the huggingface provider except it does not actually perform any
network calls and just produces constant results. It can be used to prototype
feedback function wiring for your apps before invoking potentially slow (to
run/to load) feedback functions.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.resolve()))

In [2]:
import threading

from examples.frameworks.custom.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import FeedbackMode
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.feedback.provider.hugs import Huggingface
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.resolve()
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Deleted 2 rows.
Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.16.101.32:8501 .


In [3]:
# hugs = Huggingface()
hugs = Dummy()

f_positive_sentiment = Feedback(hugs.positive_sentiment).on_output()

✅ In positive_sentiment, input text will be set to *.__record__.main_output or `Select.RecordOutput` .


In [4]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedback_mode=FeedbackMode.WITH_APP
    feedbacks=[f_positive_sentiment]
)

In [5]:
# Test of a sequence of records:
"""
with ta:
    for i, q in enumerate(["hello there"] * 100):
        print(f"\rrequest {i} ", end="")
        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")

        res = ca.respond_to_query(input=q)
"""

'\nwith ta:\n    for i, q in enumerate(["hello there"] * 100):\n        print(f"\rrequest {i} ", end="")\n        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")\n\n        res = ca.respond_to_query(input=q)\n'

In [6]:
from ipywidgets import interact
from ipywidgets import widgets
from IPython.display import JSON

In [20]:
from trulens_eval.utils.serial import JSONPath


main_input = widgets.Text(continuous_update=False)

app_selector = widgets.Text(continuous_update=False)
record_selector = widgets.Text(continuous_update=False)

display_bottom = widgets.Output()
display_top = widgets.Output()
display_side = widgets.Output()
display_middle = widgets.Output()

display_records = []

with display_bottom:
    display(main_input)
    display(app_selector)
    display(record_selector)

#with display_side:   
#with display_middle:

records = []

app_selections = []
record_selections = []

def update_record(i):
    d = display_records[i]
    record = records[i]

    d.clear_output(wait=True)

    with d:
        display(
            widgets.HTML(f"<div><b>human:</b> {record.main_input}</div>")
        )

    with d:
        display(
            widgets.HTML(f"<div><b>computer:</b> {record.main_output}</div>")
        )
        # add feedbacks here as they come in

    for s in record_selections:
        q = JSONPath.of_string(s)
        try:
            v = str(next(iter(q(record.dict()))))[0:16]
        except Exception as e:
            v = str(e)

        with d:
            display(widgets.HTML(f"<div><b>{q}</b>: {v}</div>"))

def update_record_selections():
    for i in range(len(records)):
        update_record(i)

def update_app_selections():
    display_side.clear_output(wait=True)

    for s in app_selections:
        q = JSONPath.of_string(s)

        try:
            v = str(next(iter(q(ta.dict()))))[0:16]
        except Exception as e:
            v = str(e)

        with display_side:
            display(widgets.HTML(f"<div><b>{q}</b><br>{v}</div>"))


def add_app_selection(ev):
    s = ev.new

    if s not in app_selections:
        app_selections.append(s)

    update_app_selections()

def add_record_selection(ev):
    s = ev.new

    if s not in record_selections:
        record_selections.append(s)

    update_record_selections()

def runit(ev):
    human = ev.new

    d = widgets.Output()
    display_records.append(d)

    with display_top:
        display(d)

    with ta as recording:
        comp = ca.respond_to_query(input=human)

    records.append(recording.get())
    update_record(len(records)-1)

    update_app_selections()


main_input.observe(runit, ["value"])
app_selector.observe(add_app_selection, ["value"])
record_selector.observe(add_record_selection, ["value"])


display(
    widgets.HBox([
        widgets.VBox([
            display_top,
            widgets.HBox([
              widgets.Label("main_input"),
              display_bottom
            ])
        ]),
        display_middle,
        display_side])
    )


Output()

Output()

Output()